### Clean, validate and categorise(VALID/INVALID) PAN card number data 

In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("PAN Number Validation Dataset.csv")
df

,Pan_Numbers
0,VGLOD3180G
1,PHOXD7232L
2,MGEPH6532A
3,JJCHK4574O
4,XTQIJ2330L
...,...
9028,ZMVA03025o
9029,TNGUY429!V
9030,SMMN33673g
9031,RLAI69795t


#### Data cleaning and preprocessing

In [5]:
#Identify and Handle missing data
df.shape

(9033, 1)

In [7]:
df.isnull().sum()

Pan_Numbers    0
dtype: int64

In [9]:
#Check for duplicates
duplicated_pan = df[df.duplicated()]
duplicated_pan

,Pan_Numbers
5150,XTP0675
5152,IPSLX475!I
5162,BPWVM28815K
5263,XVATX221!N
5264,XVATX221!N
5284,JVPYR52307F
5285,JVPYR52307F
5286,JVPYR52307F


In [11]:
df.drop_duplicates(inplace=True)
print(df[df.duplicated()])

Empty DataFrame
Columns: [Pan_Numbers]
Index: []


In [13]:
#Handle leading/Trailing white space
import re

for i in df['Pan_Numbers']:
    if re.search(r'^\s+|\s+$',i):
        print(f'"{i}"')

df['Pan_Numbers'] = df['Pan_Numbers'].str.strip()

"hyuij7902r  "
"DOURT5035Y. "
" ZPJQS1155M"
"     UQZ4822"
" KEMCQ031!F "
"MGES52860u "
"    DLBFA20911H  "


In [15]:
#Correct letter case
df['Pan_Numbers'] = df['Pan_Numbers'].str.upper()

In [17]:
cleaned_df = df

In [19]:
cleaned_df

,Pan_Numbers
0,VGLOD3180G
1,PHOXD7232L
2,MGEPH6532A
3,JJCHK4574O
4,XTQIJ2330L
...,...
9028,ZMVA03025O
9029,TNGUY429!V
9030,SMMN33673G
9031,RLAI69795T


### PAN format validation

- Number should be of 10 characters
- First five characters should be alphabetic(no adjacent characters should be same, all characters can not be in sequence)
- Next four characters should be numbers(no adjacent numbers should be same, all numbers can not be in sequence)
- The last character should be an alphabet

#### Verify if PAN number is in proper format 

In [21]:
import re
pan_data = pd.Series(cleaned_df['Pan_Numbers'])
cleaned_df['in_format'] = pan_data.str.contains(r"^[A-Z]{5}[0-9]{4}[A-Z]$")
cleaned_df

,Pan_Numbers,in_format
0,VGLOD3180G,True
1,PHOXD7232L,True
2,MGEPH6532A,True
3,JJCHK4574O,True
4,XTQIJ2330L,True
...,...,...
9028,ZMVA03025O,False
9029,TNGUY429!V,False
9030,SMMN33673G,False
9031,RLAI69795T,False


#### Function to check if two adjacent characters are same in PAN number,if yes, returns True else False

In [48]:
def are_adjacent_same(pan):
    for i in range(len(pan)-1):
        if pan[i] == pan[i+1]:
            return True
    return False
        
cleaned_df['are_adjacent_same'] = cleaned_df['Pan_Numbers'].str[:].apply(are_adjacent_same) 

#### Function to check if first five alphabets are in sequence or next four numbers are in sequence,if yes, returns True else False

In [50]:
def are_in_sequence(pan):
    for i in range(len(pan)-1):
        if ord(pan[i]) != ord(pan[i+1])-1:
            return False
    return True
        
cleaned_df['char_in_sequence'] = cleaned_df['Pan_Numbers'].str[:5].apply(are_in_sequence) 
cleaned_df['num_in_sequence'] = cleaned_df['Pan_Numbers'].str[6:10].apply(are_in_sequence)

In [75]:
final_df = cleaned_df

#### 'Category' column is assigned 'Valid PAN' / 'Invalid PAN' basec on the criteria 

In [77]:
final_df['Category'] = np.where(~final_df['are_adjacent_same'] &
                                ~final_df['char_in_sequence'] &
                                ~final_df['num_in_sequence'] &
                                final_df['in_format'],
                                'Valid PAN',
                                'Invalid PAN')

In [82]:
final_df

,Pan_Numbers,in_format,are_adjacent_same,char_in_sequence,num_in_sequence,Category
0,VGLOD3180G,True,False,False,False,Valid PAN
1,PHOXD7232L,True,False,False,False,Valid PAN
2,MGEPH6532A,True,False,False,False,Valid PAN
3,JJCHK4574O,True,True,False,False,Invalid PAN
4,XTQIJ2330L,True,True,False,False,Invalid PAN
...,...,...,...,...,...,...
9028,ZMVA03025O,False,False,False,False,Invalid PAN
9029,TNGUY429!V,False,False,False,False,Invalid PAN
9030,SMMN33673G,False,True,False,False,Invalid PAN
9031,RLAI69795T,False,False,False,False,Invalid PAN


### Summary

In [85]:
#Number of total records processed
record_processed = len(df)
record_processed

9025

In [87]:
#Dataframe of Valid PAN
valid_pan = final_df[final_df['Category'] == 'Valid PAN']
valid_pan

,Pan_Numbers,in_format,are_adjacent_same,char_in_sequence,num_in_sequence,Category
0,VGLOD3180G,True,False,False,False,Valid PAN
1,PHOXD7232L,True,False,False,False,Valid PAN
2,MGEPH6532A,True,False,False,False,Valid PAN
5,HTJYM3835H,True,False,False,False,Valid PAN
7,HVOFE5635Y,True,False,False,False,Valid PAN
...,...,...,...,...,...,...
4995,ZJWPC0684F,True,False,False,False,Valid PAN
4996,NSXZF3296V,True,False,False,False,Valid PAN
4997,APBHZ0717P,True,False,False,False,Valid PAN
4998,MAYLS1413K,True,False,False,False,Valid PAN


In [89]:
#Dataframe of Invalid PAN
invalid_pan = final_df[final_df['Category'] == 'Invalid PAN']
invalid_pan

,Pan_Numbers,in_format,are_adjacent_same,char_in_sequence,num_in_sequence,Category
3,JJCHK4574O,True,True,False,False,Invalid PAN
4,XTQIJ2330L,True,True,False,False,Invalid PAN
6,YQTAP6661X,True,True,False,False,Invalid PAN
12,HZVMG3577F,True,True,False,False,Invalid PAN
17,TLVFA0299E,True,True,False,False,Invalid PAN
...,...,...,...,...,...,...
9028,ZMVA03025O,False,False,False,False,Invalid PAN
9029,TNGUY429!V,False,False,False,False,Invalid PAN
9030,SMMN33673G,False,True,False,False,Invalid PAN
9031,RLAI69795T,False,False,False,False,Invalid PAN


In [91]:
summary = {'Total_Processed_Records' : record_processed,
          'Total_Valid_PAN_Numbers' : len(valid_pan),
          'Total_Invalid_PAN_Numbers' : len(invalid_pan),
          'Total_Missing_Records' : 10000 - len(df)}

In [93]:
summary

{'Total_Processed_Records': 9025,
 'Total_Valid_PAN_Numbers': 3193,
 'Total_Invalid_PAN_Numbers': 5832,
 'Total_Missing_Records': 975}